In [ ]:
import pandas as pd
import calendar
import numpy as np
from sklearn.decomposition import PCA
from datetime import datetime



## Table 3 
hier komt tabel 3, maar ik was nog even bezig met het bekijken hoe zij deze data hebben verkregen

In [ ]:
# loading dataframes 

r_daily = pd.read_csv('managed_portfolios_anom_d_50.csv')

# drop all momentum factors or factors that are constructed based on momentum

momentum_list = ['r_mom', 'r_indmom', 'r_valmom', 'r_valmomprof', 'r_mom12', 'r_momrev', 'r_indmomrev']
r_daily.drop(columns=momentum_list, inplace=True)

# set date to datetime format and set the date to the index 

r_daily['date'] = pd.to_datetime(r_daily['date'])
r_daily.set_index('date', inplace=True)

# missing list = ['dur', 'divg', 'invaci', 'ipo'] -> deze worden gemaakt uit daily stock data doormiddel van een signals list?

# following the procedure in the paper, if there are observations missing we set them to 0. 

r_daily.fillna(0, inplace=True)

# create a list of factors for later analysis purposes 

factors = [col for col in r_daily.columns if col.startswith('r_')]

In [ ]:
# initialize pca model 

pca = PCA(n_components=len(factors))

# select our start date 

start_date = pd.to_datetime("1963-07-01")

# create an empty dataframe for our monthly pc returns

pc_return_df = pd.DataFrame(columns=['date'] + [f"PC{i+1}" for i in range(len(factors))])

# create our loop set up, this is actually an expanding PCA analysis. In each iteration a new month is added to the dataset and the return is computed. 

for year in range(1974, 1975):
    for mo in range(1, 13):
        # first we have to find the last month of the day. For this we use the calender function with inputs from the loop variables
        last_day = calendar.monthrange(year, mo)[1]

        # we select our new end_date variable for which the PCA analysis is done, also with inputs from our loop and the last_day variable
        end_date = pd.to_datetime(f'{year}-{mo}-{last_day}')

        # we select the datarange from our dataset (July 1963 = start_date until our defined end_date) and we fit the model
        pca_data = r_daily.loc[start_date:end_date]
        pca.fit(pca_data)

        # we extract the principal components. These principal components are put in a new dataframe for later analysis. 

        principal_components = pca.transform(pca_data)
        principal_df = pd.DataFrame(data=principal_components, index=pca_data.index, columns=[f"PC{i+1}" for i in range(len(factors))])
        principal_df.index = pd.to_datetime(principal_df.index)

        # calculating return for month t+1. If mo = 12, then year will increment with 1. 

        return_year = year + 1 if mo == 12 else year
        return_month = (mo % 12) + 1

        # creating a datetime variable for the month t+1 and storing this in our pc_return_data variable

        return_date =pd.to_datetime(f'{return_year}-{return_month}')
        pc_return_data = {'date': return_date}

        # in this loop we calculate the monthly factor returns (f) using the principal components and returns

        for f in range(len(factors)):
            # select our factor and extract its principal component from principal_df and its return from r_daily for all observations in month mo 
            pc_f = principal_df[(principal_df.index.year == year) & (principal_df.index.month == mo)][principal_df.columns[f]]
            return_f = r_daily.loc[(r_daily.index.year == year) & (r_daily.index.month == mo), factors[f]]

            # multiply the principal components with the returns and sum them up to get PC factor return for month mo 
            pc_return_f = (pc_f*return_f).sum()

            # put the calculated return in our pc_return_data variable
            pc_return_data[principal_df.columns[f]] = pc_return_f

        # concat all the dataframes in our pc_return_df (this we can use for later analysis)
        pc_return_df = pd.concat([pc_return_df, pd.DataFrame(pc_return_data, index=[0])], ignore_index=True)

print(pc_return_df)